# SDO/AIA and Solar Orbiter/EUI alignment tutorial

In this tutorial, we will

- Retrive HRI EUV 174 EUI data and AIA data using [Fido](https://docs.sunpy.org/en/stable/generated/api/sunpy.net.Fido.html#sunpy.net.Fido)
- Correct for 'jitter' in EUI images
- align aia/eui images in time (account for timing delay)
- use wcs to align in space
- for small scale, may need to apply additional correction

----

Before jumping in, we first need to import the necessary packages. The cell should have no issues running if your conda environment is setup correctly or if you're running this notebook externally via Binder.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import astropy

import sunpy
import sunpy.map
import sunpy_soar
from sunpy.net import Fido, attrs as a

import astropy.units as u

---

First let's retrieve the data using SunPy's Fido! We'll be working with a short HRIEUV time series on March 5th 2022.

(have alternative for reading in FITS files to sunpy maps?)


In [ ]:
# EUI
instrument = a.Instrument('EUI')

time = a.Time('2022-03-05 15:20:00', '2022-03-05 15:21:00')

level = a.Level(2)

product = a.soar.Product('eui-hrieuv174-image')

eui_result = Fido.search(instrument & time & level & product)

# print out the files found
print(eui_result)

Let's download the EUI files we queried for

In [ ]:
# fetch the data files
eui_files = Fido.fetch(eui_result)

# read the data files in as a sequence of SunPy maps
sunpy.map.Map(eui_files, sequence=True)

Now that we have our data as Sunpy maps, we can easily make a movie

In [ ]:
##### Movie ####

... we see some jitter. How can we correct this? Cross correlation

In [ ]:
#### cross correlate ####

Let's play the movie again

In [ ]:
#### movie ####

Great, now the EUI pointing it consistent with itself!

---

Now let's bring in AIA data! Before downloading, we have to consider the fact that EUI is at a different distance from the Sun than SDO. We have to consider this timing offset when querying for AIA data

In [ ]:
### calculate timing offset ####

# start_time_aia
# end_time_aia

Now we have everything in terms of a 1 AU detection time -- we'll use these times to query for AIA data. For comparison purposes, we'll retrieve 171 $\AA$ data since it's a similar channel as HRIEUV's 174 $\AA$ channel

In [ ]:
# AIA

# include new timing

instrument = a.Instrument('AIA')
wavelength = a.Wavelength(171*u.angstrom)  # 171 angstrom filter

aia_result = Fido.search(instrument & time & wavelength)
print(aia_result)

In [ ]:
# fetch the data files
aia_files = Fido.fetch(aia_result)

# read the data files in as a sequence of SunPy maps
sunpy.map.Map(aia_files, sequence=True)

Now we have our AIA data as SunPy maps!

In [1]:
#### movie

In order to align our AIA 171 \AA data with our EUI 174 \AA data, we'll use SunPy's reproject function:

In [ ]:
#### reproject

# show

This does a decent job! But let's say we're interested in small-scale features, let's zoom in...

In [ ]:
#### zoom in to small feature

### see that it isn't super well-aligned

On large scales, the built-in SunPy alignment looks good. But when we zoom in, we see that it's pretty off.

A way to go about fixing this is to use cross-correlation. .. not full fov ...

In [ ]:
### cross correlate

## apply shifts -- sunpy 'apply_shifts'

### zoom in

Now the alignment is even better!

---

Woohoo! We have AIA and EUI well aligned! But what if we wanted to throw IRIS data into the mix... we can retrieve IRIS data the same way we did for EUI and AIA.

In [ ]:
### pull iris data

In [ ]:
## movie

In [ ]:
### sunpy reproject to align

In [ ]:
### zoom in. well aligned?

... ensure that IRIS is well aligned we'll need to pull in AIA data at a similar channel and cross-correlate

In [ ]:
### pull aia 1600

### cross correlate w iris 1400

### zoom in -- better!

Now we have well-aligned data among 3 instruments!